In [1]:
import yaml
import numpy as np

### Loading an experiment file

In [2]:
def read_experiments(filename, verbose=True):
    with open(r''+filename) as file:
        experiments = yaml.full_load(file)
        if verbose :
            for exp, content in experiments.items():
                print(key, ":", content)
        return experiments

In [3]:
exp = read_experiments('exp.yaml', False)

In [4]:
exp

{'Experience1': {'nb_steps': 1000,
  'nb_sync': 100,
  'tasks': 3,
  'Type1': {'number': 3,
   'type': 'RegretMatching',
   'spec': {'intertia': 0.1, 'fadding': 0.2}},
  'Type2': {'number': 2, 'type': 'FictiousPlay'},
  'Type3': {'number': 3, 'type': 'Random'}},
 'Experience2': {'tasks': 3,
  'Type1': {'number': 3,
   'type': 'RegretMatching',
   'spec': {'intertia': 0.1, 'fadding': 0.2}},
  'Type2': {'number': 2, 'type': 'FictiousPlay'},
  'Type3': {'number': 1, 'type': 'Random'}}}

### Generating experiments (all combinations)

In [5]:
# Available Agent Types :
types = {0 : "RegretMatching", 1 : "FictitiousPlay", 2 : "Random", 3 : "BestResponse", 
         4 : "SpacialAdaptative"}

In [6]:
nb_vehicles = 3 # for 3 vehicles and 4 tasks, 125 possible combinations
nb_tasks = 4

In [7]:
alloc_dim = np.zeros([len(types)]*nb_vehicles)

In [8]:
alloc_dim.shape

(5, 5, 5)

In [9]:
allAlloc = [x for x,_ in np.ndenumerate(alloc_dim)]

In [10]:
len(allAlloc)

125

In [3]:
def create_experiments(nb_steps, nb_agents, nb_tasks, nb_sync=100):
    # Available Types :
    types = {0 : "RegretMatching", 1 : "FictitiousPlay", 2 : "Random", 3 : "BestResponse", 
         4 : "SpatialAdaptative"}
    alloc_dim = np.zeros([len(types)]*nb_vehicles)
    allAlloc = [x for x,_ in np.ndenumerate(alloc_dim)]
    repartitions = []
    exp = {}
    expId = 1
    for a in allAlloc:
        seen = []
        exp_dict = {}
        exp_dict['nb_sync'] = int(nb_sync)
        exp_dict['tasks'] = int(nb_tasks)
        typeId = 1
        repartition = np.zeros(len(types))
        for t in a :
            type_dict={}
            str_type = types[t]
            #print(seen)
            if str_type not in seen :
                seen.append(str_type)
                type_dict["type"] = str_type
                cnt = int((np.array(a)==t).sum())
                type_dict["number"] = cnt
                repartition[t] = cnt
                if str_type == "RegretMatching":
                    # default values
                    dict_spec = {"inertia": 0.1, "fadding" : 0.2}
                    type_dict["spec"] = dict_spec
                # save the agent params
                exp_dict["Type"+str(typeId)] = type_dict
                typeId += 1
        already_done = np.sum([1 if np.all(repartition == r) else 0 for r in repartitions])
        if already_done < 1 :
            # save the experiment setup
            exp["Experience"+str(expId)] = exp_dict
            expId += 1
            # remember done repartitions
            repartitions.append(repartition)
            #print(repartitions)
    return exp

In [12]:
exp_outp = create_experiments(100, 3, 4, nb_sync=100)

In [13]:
exp_outp

{'Experience1': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 3,
   'spec': {'inertia': 0.1, 'fadding': 0.2}}},
 'Experience2': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'FictitiousPlay', 'number': 1}},
 'Experience3': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'Random', 'number': 1}},
 'Experience4': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'BestResponse', 'number': 1}},
 'Experience5': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {'type': 'RegretMatching',
   'number': 2,
   'spec': {'inertia': 0.1, 'fadding': 0.2}},
  'Type2': {'type': 'SpatialAdaptative', 'number': 1}},
 'Experience6': {'nb_sync': 100,
  'tasks': 4,
  'Type1': {

### Saving an experiment file

In [45]:
def write_exp_file(filename, exp):
    with open(r''+str(filename), 'w') as file:
        yaml.dump(exp, file)

In [46]:
write_exp_file("exp_3vehicles_4tasks.yaml", exp_outp)

### Run an experiment from yaml file

In [3]:
import threading
import time
import logging
from Environnement import *
from Simulation import *
from Player import *
from utils import *
from Functions import *
import re

logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)) : %(message)s',
                    )


In [4]:
myexp = read_experiments("exp_3vehicles_4tasks.yaml", False)

In [5]:
print(myexp)

{'Experience1': {'Type1': {'number': 3, 'spec': {'fadding': 0.2, 'inertia': 0.1}, 'type': 'RegretMatching'}, 'nb_sync': 100, 'tasks': 4}, 'Experience10': {'Type1': {'number': 1, 'spec': {'fadding': 0.2, 'inertia': 0.1}, 'type': 'RegretMatching'}, 'Type2': {'number': 2, 'type': 'Random'}, 'nb_sync': 100, 'tasks': 4}, 'Experience11': {'Type1': {'number': 1, 'spec': {'fadding': 0.2, 'inertia': 0.1}, 'type': 'RegretMatching'}, 'Type2': {'number': 1, 'type': 'Random'}, 'Type3': {'number': 1, 'type': 'BestResponse'}, 'nb_sync': 100, 'tasks': 4}, 'Experience12': {'Type1': {'number': 1, 'spec': {'fadding': 0.2, 'inertia': 0.1}, 'type': 'RegretMatching'}, 'Type2': {'number': 1, 'type': 'Random'}, 'Type3': {'number': 1, 'type': 'SpatialAdaptative'}, 'nb_sync': 100, 'tasks': 4}, 'Experience13': {'Type1': {'number': 1, 'spec': {'fadding': 0.2, 'inertia': 0.1}, 'type': 'RegretMatching'}, 'Type2': {'number': 2, 'type': 'BestResponse'}, 'nb_sync': 100, 'tasks': 4}, 'Experience14': {'Type1': {'number'

In [6]:
def getAgent(env, id_a, type_a, fadding = 0, inertia = 0):
    if type_a == "Random":
        return RandomPlayer(env, id_a)
    if type_a == "FictitiousPlay":
        return FPPlayer(env, id_a)
    if type_a == "RegretMatching":
        return RegretMatchingPlayer(env, id_a)
    if type_a == "BestResponse":
        return BRPlayer(env, id_a)
    if type_a == "SpatialAdaptative":
        return SAPlayer(env, id_a)
    if type_a == "SpatialFictitious":
        return SpatialFictiousPlayPlayer(env, id_a)
    if type_a == "GeneralizedRegret":
        return GeneralizedRegretMatchingPlayer(env, id_a, fadding_factor = fadding, inertia_factor = inertia)

In [8]:
# run all the experiments
for exp in myexp.keys():
    #print(exp)
    nb_sync = myexp[exp]["nb_sync"]
    nb_tasks = myexp[exp]["tasks"]
    agents = []
    entrys = myexp[exp].keys()
    #print(entrys)
    res = []
    for i in entrys : # search for Type definitions only
        if len(re.findall('Type*', i)) > 0 :
            res.append(i)
    total_agents = 0
    for t in res :
        nb_t = myexp[exp][t]["number"] # number of agents of this type
        total_agents += nb_t
    #print(total_agents)
    env = Environnement(nb_tasks,total_agents) #creating the environment
    for t in res :
        if np.isin("spec",myexp[exp][t].keys()): # check if extra parameters
            fadding = myexp[exp][t]["spec"]["fadding"]
            inertia = myexp[exp][t]["spec"]["inertia"]
        else:
            fadding = 0
            inertia = 0
        for j in range(myexp[exp][t]["number"]): #create X agents of this type
            ag = getAgent(env, len(agents),myexp[exp][t]["type"], fadding, inertia)
            agents.append(ag)
    #print(agents)
    threads = []
    for a in agents : # getting agent threads
        t = threading.Thread(name="Player "+str(a.id), target=a.getWorkerFunction(10,verbose=False))
        threads.append(t)
    for th in threads : #starting the threads
        th.start()
    for th in threads : # join all
        th.join()
    #print(env.is_EN())

[DEBUG] (Player 0  ) Expected utility  [1.171875 1.578125 1.1875   0.96875 ]
[DEBUG] (Player 1  ) Expected utility  [1.       0.96875  1.328125 0.984375]
[DEBUG] (Player 2  ) Expected utility  [1.265625 1.15625  1.125    0.859375]
[DEBUG] (Player 0  ) Expected utility  [1.171875 1.578125 1.1875   0.96875 ]
[DEBUG] (Player 1  ) Expected utility  [1.       0.96875  1.328125 0.984375]
[DEBUG] (Player 2  ) Expected utility  [1.265625 1.15625  1.125    0.859375]
[DEBUG] (Player 0  ) Expected utility  [0.9375 2.525  0.95   0.775 ]
[DEBUG] (Player 1  ) Expected utility  [0.81 0.75 2.14 0.76]
[DEBUG] (Player 2  ) Expected utility  [2.032 0.952 0.8   0.688]
[DEBUG] (Player 0  ) Expected utility  [0.952 2.432 0.936 0.808]
[DEBUG] (Player 1  ) Expected utility  [0.824 0.72  2.144 0.792]
[DEBUG] (Player 2  ) Expected utility  [2.032 0.952 0.8   0.688]
[DEBUG] (Player 0  ) Expected utility  [0.79333333 3.04       0.78       0.67333333]
[DEBUG] (Player 1  ) Expected utility  [0.7        0.57777778 2

[DEBUG] (Player 1  ) Expected utility  [4.81161695 0.47095761 0.10871272 0.32299843]
[DEBUG] (Player 0  ) Expected utility  [0.9375  0.96875 0.9375  1.125  ]
[DEBUG] (Player 1  ) Expected utility  [1.515625 1.109375 1.296875 1.125   ]
[DEBUG] (Player 0  ) Expected utility  [0.98214286 1.125      0.9375     1.21428571]
[DEBUG] (Player 1  ) Expected utility  [1.54910714 1.16517857 0.90625    0.76785714]
[DEBUG] (Player 0  ) Expected utility  [0.78571429 0.9        0.75       1.94285714]
[DEBUG] (Player 1  ) Expected utility  [2.62285714 1.04857143 0.7        0.67714286]
[DEBUG] (Player 0  ) Expected utility  [0.65714286 0.80285714 0.73428571 2.04571429]
[DEBUG] (Player 1  ) Expected utility  [2.62285714 1.04857143 0.7        0.67714286]
[DEBUG] (Player 0  ) Expected utility  [0.54761905 0.66904762 0.61190476 2.55714286]
[DEBUG] (Player 1  ) Expected utility  [3.39880952 0.93849206 0.56944444 0.59920635]
[DEBUG] (Player 0  ) Expected utility  [0.47619048 0.61507937 0.6031746  2.64285714]


[DEBUG] (Player 2  ) Expected utility  [0.38265306 0.22321429 4.44260204 0.20663265]
[DEBUG] (Player 2  ) Expected utility  [0.38265306 0.22321429 4.44260204 0.20663265]
[DEBUG] (Player 2  ) Expected utility  [0.35321821 0.20604396 4.55651491 0.19073783]
[DEBUG] (Player 2  ) Expected utility  [0.35321821 0.20604396 4.55651491 0.19073783]
[DEBUG] (Player 0  ) Expected utility  [1.46875  1.296875 1.03125  0.953125]
[DEBUG] (Player 0  ) Expected utility  [1.07653061 1.29209184 1.07780612 1.03061224]
[DEBUG] (Player 0  ) Expected utility  [0.86122449 2.06734694 0.8622449  0.8244898 ]
[DEBUG] (Player 0  ) Expected utility  [0.86122449 2.06734694 0.8622449  0.8244898 ]
[DEBUG] (Player 0  ) Expected utility  [0.71768707 2.58418367 0.71853741 0.68707483]
[DEBUG] (Player 0  ) Expected utility  [0.71768707 2.58418367 0.71853741 0.68707483]
[DEBUG] (Player 0  ) Expected utility  [0.61516035 2.95335277 0.61588921 0.58892128]
[DEBUG] (Player 0  ) Expected utility  [0.61516035 2.95335277 0.61588921 

[DEBUG] (Player 0  ) Expected utility  [0.19438776 0.31479592 0.23214286 5.41071429]
[DEBUG] (Player 0  ) Expected utility  [0.17671614 0.28617811 0.21103896 5.62152134]
[DEBUG] (Player 0  ) Expected utility  [0.17671614 0.28617811 0.21103896 5.62152134]
[DEBUG] (Player 0  ) Expected utility  [0.1619898  0.26232993 0.19345238 5.79719388]
[DEBUG] (Player 0  ) Expected utility  [0.1619898  0.26232993 0.19345238 5.79719388]
[DEBUG] (Player 0  ) Expected utility  [0.14952904 0.24215071 0.17857143 5.94583987]
[DEBUG] (Player 0  ) Expected utility  [0.14952904 0.24215071 0.17857143 5.94583987]
[DEBUG] (Player 0  ) Expected utility  [1.171875 1.28125  1.1875   1.09375 ]
[DEBUG] (Player 0  ) Expected utility  [0.52933673 1.80102041 1.80612245 1.64540816]
[DEBUG] (Player 0  ) Expected utility  [0.42346939 1.44081633 2.88979592 1.31632653]
[DEBUG] (Player 0  ) Expected utility  [0.42346939 1.44081633 2.88979592 1.31632653]
[DEBUG] (Player 0  ) Expected utility  [0.35289116 1.20068027 3.6122449  

[DEBUG] (Player 1  ) Expected utility  [0.20181406 0.18537415 4.15646259 0.42120181]
[DEBUG] (Player 1  ) Expected utility  [0.20181406 0.18537415 4.15646259 0.42120181]
[DEBUG] (Player 1  ) Expected utility  [0.18163265 0.16683673 4.36428571 0.37908163]
[DEBUG] (Player 1  ) Expected utility  [0.18163265 0.16683673 4.36428571 0.37908163]
[DEBUG] (Player 1  ) Expected utility  [0.16512059 0.15166976 4.53432282 0.34461967]
[DEBUG] (Player 1  ) Expected utility  [0.16512059 0.15166976 4.53432282 0.34461967]
[DEBUG] (Player 1  ) Expected utility  [0.15136054 0.13903061 4.67602041 0.31590136]
[DEBUG] (Player 1  ) Expected utility  [0.15136054 0.13903061 4.67602041 0.31590136]
[DEBUG] (Player 1  ) Expected utility  [0.13971743 0.12833595 4.79591837 0.29160126]
[DEBUG] (Player 1  ) Expected utility  [0.13971743 0.12833595 4.79591837 0.29160126]
[DEBUG] (Player 1  ) Expected utility  [1.03125    1.07291667 1.11458333 1.15625   ]
[DEBUG] (Player 1  ) Expected utility  [1.13636364 1.03636364 1.2